In [1]:
import pandas as pd
import numpy as np
import math as mt
import os.path
import copy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

<h1><b><font color=blue>Data Extraction Diagram: </font></b></h1>

---

<center><img src="https://github.com/omidkj/IMAGES/blob/master/Screen%20Shot%202018-12-11%20at%205.48.35%20PM.png?raw=true" alt="Feature Eng" width="900" height="1200"/></center>

<h3><b><font color=blue> Query MIMIC-III dataset: join diagnoses_icd and noteevents using ICD codes as a filter and Discharge summary notes. then save the results as  </font></b></h3>

---

In [2]:
data = pd.read_csv("diagnoses_icd_final.csv", header = 0, encoding = "utf_16", sep = "\t")

In [3]:
data.head()

,Number of Records,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Hadm Id,Icd9 Code,Iserror,Row Id (Noteevents),Row Id,Seq Num,Storetime,Subject Id (Noteevents),Subject Id,Text
0,1,Discharge summary,NaN,6/14/2118 12:00:00 AM,NaN,Report,107527,107527,486,NaN,175,153655,4,NaN,13702,13702,Admission Date: [**2118-6-2**] Discharg...
1,1,Discharge summary,NaN,6/14/2118 12:00:00 AM,NaN,Report,107527,107527,49121,NaN,175,153653,2,NaN,13702,13702,Admission Date: [**2118-6-2**] Discharg...
2,1,Discharge summary,NaN,5/25/2119 12:00:00 AM,NaN,Report,167118,167118,496,NaN,176,153661,3,NaN,13702,13702,Admission Date: [**2119-5-4**] D...
3,1,Discharge summary,NaN,8/18/2124 12:00:00 AM,NaN,Report,196489,196489,4160,NaN,177,153670,6,NaN,13702,13702,Admission Date: [**2124-7-21**] ...
4,1,Discharge summary,NaN,8/18/2124 12:00:00 AM,NaN,Report,196489,196489,49121,NaN,177,153668,4,NaN,13702,13702,Admission Date: [**2124-7-21**] ...


In [4]:
data.shape

(48153, 17)

In [5]:
data[data['Icd9 Code'] == 4172]['Icd9 Code'].value_counts()

Series([], Name: Icd9 Code, dtype: int64)

In [1]:
#data['Icd9 Code'].value_counts()

In [7]:
data.dtypes

Number of Records            int64
Category                    object
Cgid                       float64
Chartdate                   object
Charttime                  float64
Description                 object
Hadm Id (Noteevents)         int64
Hadm Id                      int64
Icd9 Code                    int64
Iserror                    float64
Row Id (Noteevents)          int64
Row Id                       int64
Seq Num                      int64
Storetime                  float64
Subject Id (Noteevents)      int64
Subject Id                   int64
Text                        object
dtype: object

<h3><b><font color=blue> Keeping the last Discharge note for each patient  </font></b></h3>

---

In [8]:
data.sort_values(by=['Hadm Id','Row Id (Noteevents)'])

,Number of Records,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Hadm Id,Icd9 Code,Iserror,Row Id (Noteevents),Row Id,Seq Num,Storetime,Subject Id (Noteevents),Subject Id,Text
8893,1,Discharge summary,NaN,4/17/2108 12:00:00 AM,NaN,Report,100006,100006,486,NaN,8772,110909,3,NaN,9895,9895,Admission Date: [**2108-4-6**] Discharg...
8894,1,Discharge summary,NaN,4/17/2108 12:00:00 AM,NaN,Report,100006,100006,49320,NaN,8772,110907,1,NaN,9895,9895,Admission Date: [**2108-4-6**] Discharg...
45126,1,Discharge summary,NaN,4/18/2108 12:00:00 AM,NaN,Addendum,100006,100006,486,NaN,55972,110909,3,NaN,9895,9895,"Name: [**Known lastname 470**], [**Known firs..."
45127,1,Discharge summary,NaN,4/18/2108 12:00:00 AM,NaN,Addendum,100006,100006,49320,NaN,55972,110907,1,NaN,9895,9895,"Name: [**Known lastname 470**], [**Known firs..."
34530,1,Discharge summary,NaN,4/7/2145 12:00:00 AM,NaN,Report,100007,100007,486,NaN,50238,256428,4,NaN,23018,23018,Admission Date: [**2145-3-31**] ...
22288,1,Discharge summary,NaN,9/12/2177 12:00:00 AM,NaN,Report,100011,100011,48242,NaN,39781,596527,4,NaN,87977,87977,Admission Date: [**2177-8-29**] ...
11329,1,Discharge summary,NaN,9/6/2176 12:00:00 AM,NaN,Report,100018,100018,496,NaN,16580,462394,25,NaN,58128,58128,Admission Date: [**2176-8-29**] ...
11330,1,Discharge summary,NaN,9/6/2176 12:00:00 AM,NaN,Report,100018,100018,4280,NaN,16580,462383,14,NaN,58128,58128,Admission Date: [**2176-8-29**] ...
11331,1,Discharge summary,NaN,9/6/2176 12:00:00 AM,NaN,Report,100018,100018,42833,NaN,16580,462373,4,NaN,58128,58128,Admission Date: [**2176-8-29**] ...
8111,1,Discharge summary,NaN,12/10/2142 12:00:00 AM,NaN,Report,100020,100020,4280,NaN,6876,111824,7,NaN,9973,9973,Admission Date: [**2142-11-30**] ...


In [9]:
data_2 = data.loc[data['Description'] != 'Addendum']

In [10]:
data_2 = (data_2.groupby(['Hadm Id']).nth(-1)).reset_index()    

In [11]:
data_2.reset_index(drop=True)

,Hadm Id,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Icd9 Code,Iserror,Number of Records,Row Id,Row Id (Noteevents),Seq Num,Storetime,Subject Id,Subject Id (Noteevents),Text
0,100006,Discharge summary,NaN,4/17/2108 12:00:00 AM,NaN,Report,100006,49320,NaN,1,110907,8772,1,NaN,9895,9895,Admission Date: [**2108-4-6**] Discharg...
1,100007,Discharge summary,NaN,4/7/2145 12:00:00 AM,NaN,Report,100007,486,NaN,1,256428,50238,4,NaN,23018,23018,Admission Date: [**2145-3-31**] ...
2,100011,Discharge summary,NaN,9/12/2177 12:00:00 AM,NaN,Report,100011,48242,NaN,1,596527,39781,4,NaN,87977,87977,Admission Date: [**2177-8-29**] ...
3,100018,Discharge summary,NaN,9/6/2176 12:00:00 AM,NaN,Report,100018,42833,NaN,1,462373,16580,4,NaN,58128,58128,Admission Date: [**2176-8-29**] ...
4,100020,Discharge summary,NaN,12/10/2142 12:00:00 AM,NaN,Report,100020,4280,NaN,1,111824,6876,7,NaN,9973,9973,Admission Date: [**2142-11-30**] ...
5,100030,Discharge summary,NaN,12/19/2199 12:00:00 AM,NaN,Report,100030,41519,NaN,1,144135,9785,1,NaN,12803,12803,Admission Date: [**2199-12-3**] Dischar...
6,100035,Discharge summary,NaN,3/19/2115 12:00:00 AM,NaN,Report,100035,49391,NaN,1,417501,41331,1,NaN,48539,48539,Admission Date: [**2115-2-22**] ...
7,100036,Discharge summary,NaN,7/23/2187 12:00:00 AM,NaN,Report,100036,42833,NaN,1,344884,13877,2,NaN,30078,30078,Admission Date: [**2187-7-13**] ...
8,100037,Discharge summary,NaN,5/9/2183 12:00:00 AM,NaN,Report,100037,486,NaN,1,466926,13143,4,NaN,58947,58947,Admission Date: [**2183-3-23**] ...
9,100039,Discharge summary,NaN,5/17/2174 12:00:00 AM,NaN,Report,100039,42843,NaN,1,438846,43828,2,NaN,53014,53014,Admission Date: [**2174-4-18**] ...


In [12]:
data_2.shape

(23907, 17)

In [13]:
data_2.head()

,Hadm Id,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Icd9 Code,Iserror,Number of Records,Row Id,Row Id (Noteevents),Seq Num,Storetime,Subject Id,Subject Id (Noteevents),Text
0,100006,Discharge summary,NaN,4/17/2108 12:00:00 AM,NaN,Report,100006,49320,NaN,1,110907,8772,1,NaN,9895,9895,Admission Date: [**2108-4-6**] Discharg...
1,100007,Discharge summary,NaN,4/7/2145 12:00:00 AM,NaN,Report,100007,486,NaN,1,256428,50238,4,NaN,23018,23018,Admission Date: [**2145-3-31**] ...
2,100011,Discharge summary,NaN,9/12/2177 12:00:00 AM,NaN,Report,100011,48242,NaN,1,596527,39781,4,NaN,87977,87977,Admission Date: [**2177-8-29**] ...
3,100018,Discharge summary,NaN,9/6/2176 12:00:00 AM,NaN,Report,100018,42833,NaN,1,462373,16580,4,NaN,58128,58128,Admission Date: [**2176-8-29**] ...
4,100020,Discharge summary,NaN,12/10/2142 12:00:00 AM,NaN,Report,100020,4280,NaN,1,111824,6876,7,NaN,9973,9973,Admission Date: [**2142-11-30**] ...


In [2]:
#data_2['Icd9 Code'].value_counts()

In [3]:
 #data_2[data_2["Hadm Id"] == 113071]

<h3><b><font color=blue> Choose a sample of the dataset to be processed in NLP model with the ratio of 0.2: </font></b></h3>

---

In [16]:
#chosen_idx = np.random.choice(23907, replace=False, size=4500, random=110)
df_trimmed = data_2.sample(frac=0.2, weights='Icd9 Code',random_state=110)

In [4]:
#df_trimmed['Icd9 Code'].value_counts()

In [18]:
df_trimmed.head()

,Hadm Id,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Icd9 Code,Iserror,Number of Records,Row Id,Row Id (Noteevents),Seq Num,Storetime,Subject Id,Subject Id (Noteevents),Text
2842,111544,Discharge summary,NaN,4/3/2179 12:00:00 AM,NaN,Report,111544,49390,NaN,1,337578,29750,16,NaN,29514,29514,Admission Date: [**2179-3-22**] ...
15822,166737,Discharge summary,NaN,3/15/2161 12:00:00 AM,NaN,Report,166737,41519,NaN,1,466894,42433,2,NaN,58939,58939,Admission Date: [**2161-3-10**] ...
8980,137804,Discharge summary,NaN,6/1/2179 12:00:00 AM,NaN,Report,137804,49390,NaN,1,545789,49730,15,NaN,76479,76479,Admission Date: [**2179-5-23**] ...
14743,162201,Discharge summary,NaN,10/3/2197 12:00:00 AM,NaN,Report,162201,42823,NaN,1,369318,50123,2,NaN,32074,32074,Admission Date: [**2197-9-27**] ...
16320,168769,Discharge summary,NaN,2/16/2198 12:00:00 AM,NaN,Report,168769,42820,NaN,1,129266,48454,6,NaN,11486,11486,Admission Date: [**2198-1-19**] ...


In [ ]:
df_trimmed = df_trimmed.reset_index(drop=True)

In [ ]:
df_trimmed.head()

,Hadm Id,Category,Cgid,Chartdate,Charttime,Description,Hadm Id (Noteevents),Icd9 Code,Iserror,Number of Records,Row Id,Row Id (Noteevents),Seq Num,Storetime,Subject Id,Subject Id (Noteevents),Text
0,111544,Discharge summary,NaN,4/3/2179 12:00:00 AM,NaN,Report,111544,49390,NaN,1,337578,29750,16,NaN,29514,29514,Admission Date: [**2179-3-22**] ...
1,166737,Discharge summary,NaN,3/15/2161 12:00:00 AM,NaN,Report,166737,41519,NaN,1,466894,42433,2,NaN,58939,58939,Admission Date: [**2161-3-10**] ...
2,137804,Discharge summary,NaN,6/1/2179 12:00:00 AM,NaN,Report,137804,49390,NaN,1,545789,49730,15,NaN,76479,76479,Admission Date: [**2179-5-23**] ...
3,162201,Discharge summary,NaN,10/3/2197 12:00:00 AM,NaN,Report,162201,42823,NaN,1,369318,50123,2,NaN,32074,32074,Admission Date: [**2197-9-27**] ...
4,168769,Discharge summary,NaN,2/16/2198 12:00:00 AM,NaN,Report,168769,42820,NaN,1,129266,48454,6,NaN,11486,11486,Admission Date: [**2198-1-19**] ...


In [ ]:
#test_df = df_trimmed.iloc[0:10,:]

<h3><b><font color=blue> Cleaning texts from newlines and tabs to make it ready to be processed in NLP model </font></b></h3>

---

In [ ]:
#test_df.Text = test_df.Text.fillna(' ')
df_trimmed.Text = df_trimmed.Text.str.replace('\n',' ')
df_trimmed.Text = df_trimmed.Text.str.replace('\t',' ')

<h3><b><font color=blue> Saving the final sample dataset as a CSV file to be processed in the NLP pipeline:  </font></b></h3>

---

In [ ]:
df_trimmed.to_csv('noLine_data.csv', sep="\t")

file_name = ""
save_path = "./input"
for i in range(len(test_df)):
    if i != 0:
        file_name = (5-int(mt.log10(i)))*"0" + str(i)
    else:
        file_name = "000000"
    completeName = os.path.join(save_path, file_name+".txt")         
    f = open(completeName, "w+")
    f.write(test_df['Text'][i])
    f.close()